In [1]:
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import keras.layers as layers
import spacy
from gensim.corpora import Dictionary
from keras.models import Model
from keras.preprocessing import sequence
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from spacy.util import minibatch
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle


outlets = pd.read_csv('outlets.csv')
items = pd.read_csv('items.csv')
#ireplies = pd.read_csv('ireplies.csv')

dfaux = pd.merge(outlets, items, on = 'newsoutlet_id', how='inner')
#df = pd.merge(dfaux, ireplies, left_on='news_id', right_on='in_reply_to_news_id')
dfaux = dfaux[0:5]
dfaux.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   newsoutlet_id       5 non-null      int64 
 1   newsoutlet_name     5 non-null      object
 2   newsoutlet_country  5 non-null      object
 3   news_id             5 non-null      int64 
 4   news_tweet_date     5 non-null      object
 5   news_url            5 non-null      object
 6   new_title           5 non-null      object
 7   news_date           0 non-null      object
 8   news_text           5 non-null      object
dtypes: int64(2), object(7)
memory usage: 400.0+ bytes


In [2]:
dfaux.iloc[0]

newsoutlet_id                                                    428333
newsoutlet_name                                     "CNN Breaking News"
newsoutlet_country                                                  USA
news_id                                             1080369723227979776
news_tweet_date                                     2019-01-02 07:46:28
news_url                                         https://cnn.it/2AmMvDy
new_title             Winning ticket sold for $425 million New Year'...
news_date                                                           NaN
news_text             (CNN) Another year, another lottery you didn't...
Name: 0, dtype: object

In [3]:
def pipe_dropColumns(data):
    return data.drop(columns=['news_url', 'news_date', 'newsoutlet_id'])

In [4]:
dfaux.pipe(pipe_dropColumns)

,newsoutlet_name,newsoutlet_country,news_id,news_tweet_date,new_title,news_text
0,"""CNN Breaking News""",USA,1080369723227979776,2019-01-02 07:46:28,Winning ticket sold for $425 million New Year'...,"(CNN) Another year, another lottery you didn't..."
1,"""CNN Breaking News""",USA,1078295012532736001,2018-12-27 14:22:19,Clashes in Tunisia after journalist sets himse...,(CNN) Clashes in Tunisia have continued betwee...
2,"""CNN Breaking News""",USA,1079909388524154880,2019-01-01 01:17:16,"Chief Justice Roberts avoids controversy, prai...",(CNN) — Chief Justice John Roberts steered cle...
3,"""CNN Breaking News""",USA,1081296207790538753,2019-01-04 21:08:00,Mueller grand jury gets 6 month extension,Washington (CNN) Special counsel Robert Muelle...
4,"""CNN Breaking News""",USA,1081777463351541762,2019-01-06 05:00:20,Jazmine Barnes shooting: Police interview 'per...,(CNN) Detectives investigating the drive-by mu...


### Title entities

In [5]:
import string
import en_core_web_sm

nlp = en_core_web_sm.load()

title_ent = list(["PERSON", "ORG", "GPE", "DATE", "CARDINAL", 
                  "NORP", "EVENT", "ORDINAL", "MONEY", "LOC", 
                  "WORK_OF_ART", "FAC", "TIME", "PRODUCT", "QUANTITY", 
                  "PERCENT", "LANGUAGE", "LAW"])

def to_entities(text):
    '''
    Recebe um texto e devolve uma lista com as entidades (entities) desse texto
    
    '''
    doc = nlp(text)
    return[X.label_ for X in doc.ents]

In [6]:
def pipe_title_entities(data):
    # Retira as entidades de cada título para uma lista (Nova coluna: 'title_entities')
    listsOfEntities_S = data['new_title'].apply(lambda x: to_entities(x))
    
    # Adiciona linha à Serie listsOfEntities_S com a lista de todas as entidades
    # para as criar todas no OHE e no concat do return retira essa última linha
    auxS = pd.Series([title_ent])
    listsOfEntities_S = listsOfEntities_S.append(auxS, ignore_index=True)

    # 'title_entities' lists to String
    strTtlEnt = listsOfEntities_S.apply(', '.join)

    # 'title_entities' OHE
    oheTtlEnt = strTtlEnt.str.get_dummies(sep=', ').add_prefix('title_')
    
    return pd.concat([data, oheTtlEnt[:-1]], axis=1)

In [7]:
pipe1 = dfaux.pipe(pipe_dropColumns).pipe(pipe_title_entities)
pipe1

,newsoutlet_name,newsoutlet_country,news_id,news_tweet_date,new_title,news_text,title_CARDINAL,title_DATE,title_EVENT,title_FAC,...,title_MONEY,title_NORP,title_ORDINAL,title_ORG,title_PERCENT,title_PERSON,title_PRODUCT,title_QUANTITY,title_TIME,title_WORK_OF_ART
0,"""CNN Breaking News""",USA,1080369723227979776,2019-01-02 07:46:28,Winning ticket sold for $425 million New Year'...,"(CNN) Another year, another lottery you didn't...",0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,"""CNN Breaking News""",USA,1078295012532736001,2018-12-27 14:22:19,Clashes in Tunisia after journalist sets himse...,(CNN) Clashes in Tunisia have continued betwee...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"""CNN Breaking News""",USA,1079909388524154880,2019-01-01 01:17:16,"Chief Justice Roberts avoids controversy, prai...",(CNN) — Chief Justice John Roberts steered cle...,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,"""CNN Breaking News""",USA,1081296207790538753,2019-01-04 21:08:00,Mueller grand jury gets 6 month extension,Washington (CNN) Special counsel Robert Muelle...,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,"""CNN Breaking News""",USA,1081777463351541762,2019-01-06 05:00:20,Jazmine Barnes shooting: Police interview 'per...,(CNN) Detectives investigating the drive-by mu...,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### Text entities

In [8]:
def pipe_text_entities(data):
    # Retira as entidades de cada texto noticioso para uma lista (Nova coluna: 'text_entities')
    listsOfEntities_S = data['news_text'].apply(lambda x: to_entities(x))

    # Adiciona linha à Serie listsOfEntities_S com a lista de todas as entidades
    # para as criar todas no OHE e no concat do return retira essa última linha
    auxS = pd.Series([title_ent]) # lista completa de entidades é igual para texto e título
    listsOfEntities_S = listsOfEntities_S.append(auxS, ignore_index=True)
    
    # 'text_entities' lists to String
    strTxtEnt = listsOfEntities_S.apply(', '.join)

    # 'text_entities' OHE
    oheTxtEnt = strTxtEnt.str.get_dummies(sep=', ').add_prefix('text_')
        
    return pd.concat([data, oheTxtEnt[:-1]], axis=1)

In [9]:
pipe2 = pipe1.pipe(pipe_text_entities)
pipe2

,newsoutlet_name,newsoutlet_country,news_id,news_tweet_date,new_title,news_text,title_CARDINAL,title_DATE,title_EVENT,title_FAC,...,text_MONEY,text_NORP,text_ORDINAL,text_ORG,text_PERCENT,text_PERSON,text_PRODUCT,text_QUANTITY,text_TIME,text_WORK_OF_ART
0,"""CNN Breaking News""",USA,1080369723227979776,2019-01-02 07:46:28,Winning ticket sold for $425 million New Year'...,"(CNN) Another year, another lottery you didn't...",0,0,1,0,...,1,0,1,1,0,1,0,0,0,0
1,"""CNN Breaking News""",USA,1078295012532736001,2018-12-27 14:22:19,Clashes in Tunisia after journalist sets himse...,(CNN) Clashes in Tunisia have continued betwee...,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0
2,"""CNN Breaking News""",USA,1079909388524154880,2019-01-01 01:17:16,"Chief Justice Roberts avoids controversy, prai...",(CNN) — Chief Justice John Roberts steered cle...,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
3,"""CNN Breaking News""",USA,1081296207790538753,2019-01-04 21:08:00,Mueller grand jury gets 6 month extension,Washington (CNN) Special counsel Robert Muelle...,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
4,"""CNN Breaking News""",USA,1081777463351541762,2019-01-06 05:00:20,Jazmine Barnes shooting: Police interview 'per...,(CNN) Detectives investigating the drive-by mu...,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0


### Title Entities Frequency

### V1

In [10]:
def tuple_word_ent(txt):
    '''
    Recebe um texto e devolve uma lista de pares com (substantivo, entidade)
    
    '''
    doc = nlp(txt)
    return[(X.text,X.label_) for X in doc.ents]

In [11]:
def pipe_TtlEnt_freqV1(data):
    data['tuple_title_entities'] = data['new_title'].apply(lambda x: tuple_word_ent(x))
    # Cria colunas com o nome de cada tipo de entidade 
    # e preenche com zeros todas as linhas
    for ent_type in title_ent:
        data['freq_' + ent_type] = 0
    
    # preenche com a frequência de cada entidade
    for index, row in data.iterrows():
        for (word,ent) in row['tuple_title_entities']:
            data['freq_' + ent].iloc[index] += 1
            
    return data.drop(columns=['tuple_title_entities'])

In [12]:
pipe3_V1 = pipe2.pipe(pipe_TtlEnt_freqV1)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


### V2

In [13]:
top200_nouns = []

with open('top200_nouns.data', 'rb') as filehandle:
    top200_nouns = pickle.load(filehandle)


def pipe_TtlEnt_freqV2(data):
    # Cria colunas com o nome de cada tipo de entidade 
    # e preenche com zeros todas as linhas
    for ent_type in top200_nouns:
        data['noun_freq_' + ent_type] = 0

    # preenche com a frequência de cada entidade
    for index, row in data.iterrows():
        for (word,ent) in row['tuple_title_entities']:
            if word in top200_nouns:
                data['noun_freq_' + word].iloc[index] += 1
    return data.drop(columns=['tuple_title_entities'])

In [14]:
pipe3_V2 = pipe2.pipe(pipe_TtlEnt_freqV2)

In [15]:
pipe3_V2.head(3)

,newsoutlet_name,newsoutlet_country,news_id,news_tweet_date,new_title,news_text,title_CARDINAL,title_DATE,title_EVENT,title_FAC,...,noun_freq_Michelle Obama,noun_freq_Luther,noun_freq_Jeremy Clarkson,noun_freq_Bob Einstein,noun_freq_Hillary Clinton,noun_freq_Michael Schumacher,noun_freq_Kentucky,noun_freq_Georgia,noun_freq_Venezuela,noun_freq_TSA
0,"""CNN Breaking News""",USA,1080369723227979776,2019-01-02 07:46:28,Winning ticket sold for $425 million New Year'...,"(CNN) Another year, another lottery you didn't...",0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,"""CNN Breaking News""",USA,1078295012532736001,2018-12-27 14:22:19,Clashes in Tunisia after journalist sets himse...,(CNN) Clashes in Tunisia have continued betwee...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"""CNN Breaking News""",USA,1079909388524154880,2019-01-01 01:17:16,"Chief Justice Roberts avoids controversy, prai...",(CNN) — Chief Justice John Roberts steered cle...,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### Title keywords

In [16]:
from rake_nltk import Rake

def get_keywords(text):
    r = Rake(punctuations=string.punctuation+'’‘')
    r.extract_keywords_from_text(text)
    
    return r.get_ranked_phrases()

In [17]:
# Carrega a lista com as 1000 keywords mais frequentes do dataset
with open('top1000_kw.data', 'rb') as filehandle:
    top1000_kw = pickle.load(filehandle)

def filterKws(kw_list):
    return [i for i in kw_list if i in top1000_kw]

In [18]:
# A cada texto apenas extrai as keywords que pertencem às 1000 mais frequentes
def keywordsExt(text):
    r = Rake(punctuations=string.punctuation+'’‘')
    r.extract_keywords_from_text(text)
    kw_list = r.get_ranked_phrases()

    return filterKws(kw_list)

In [19]:
with open('bow_transformer.pkl', 'rb') as f:
    bow_transformer = pickle.load(f)


def pipe_ttl_keywords(data):
    title_bow = bow_transformer.transform(data['new_title'])
    ttl_kw_bow = pd.DataFrame.sparse.from_spmatrix(title_bow)
    data = pd.merge(data, ttl_kw_bow, left_index=True, right_index=True)
    return data

In [20]:
pipe4 = pipe3_V1.pipe(pipe_ttl_keywords)

In [21]:
pipe4.head(3)

,newsoutlet_name,newsoutlet_country,news_id,news_tweet_date,new_title,news_text,title_CARDINAL,title_DATE,title_EVENT,title_FAC,...,990,991,992,993,994,995,996,997,998,999
0,"""CNN Breaking News""",USA,1080369723227979776,2019-01-02 07:46:28,Winning ticket sold for $425 million New Year'...,"(CNN) Another year, another lottery you didn't...",0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,"""CNN Breaking News""",USA,1078295012532736001,2018-12-27 14:22:19,Clashes in Tunisia after journalist sets himse...,(CNN) Clashes in Tunisia have continued betwee...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"""CNN Breaking News""",USA,1079909388524154880,2019-01-01 01:17:16,"Chief Justice Roberts avoids controversy, prai...",(CNN) — Chief Justice John Roberts steered cle...,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


### Topic Category

In [22]:
MAX_SEQUENCE_LEN = 40
UNK = 'UNK' # unknown tokens
PAD = 'PAD' # padding tokens

def text_to_id_list(text, dictionary):
    return [dictionary.token2id.get(tok, dictionary.token2id.get(UNK))
            for tok in text_to_tokens(text)]

def texts_to_input(texts, dictionary):
    return sequence.pad_sequences(
        list(map(lambda x: text_to_id_list(x, dictionary), texts)), maxlen=MAX_SEQUENCE_LEN,
        padding='post', truncating='post', value=dictionary.token2id.get(PAD))

def text_to_tokens(text):
    return [tok.text.lower() for tok in nlp.tokenizer(text)
            if not (tok.is_punct or tok.is_quote)]

In [23]:
import tensorflow as tf
from tensorflow import keras

with open('TC_dictionary.data', 'rb') as filehandle:
    dictionary = pickle.load(filehandle)

with open('labelbinarizer.data', 'rb') as filehandle:
    lb = pickle.load(filehandle)
    

def pipe_topic_category(data):
    model = tf.keras.models.load_model('CNN_topic_category_topCats')
    nlp = spacy.load('en_core_web_sm')
    ttl2input = texts_to_input(data['new_title'], dictionary)
    
    prob_cats = model.predict(ttl2input)
    
    predicted_cats = list()
    inf_limit = 0.7
    for i in prob_cats:
        if np.amax(i) >= inf_limit:
            predicted_cats.append(lb.classes_[np.argmax(i)])
        else:
            predicted_cats.append('OTHERS')
    
    # Cria uma Serie com as categorias previstas mais todas as 10 categorias
    # para criar o OHE com todas as categorias e remove as linhas a mais no merge do return
    categories_list = list(lb.classes_)
    categories_list.append('OTHERS')
    pred_cats_S = pd.Series(predicted_cats+categories_list)
    categories_df = pd.get_dummies(pred_cats_S, prefix='TC_')

    return pd.merge(data, categories_df[:-10], left_index=True, right_index=True)

In [24]:
PipeNewsContent = pipe4.pipe(pipe_topic_category)

In [25]:
PipeNewsContent

,newsoutlet_name,newsoutlet_country,news_id,news_tweet_date,new_title,news_text,title_CARDINAL,title_DATE,title_EVENT,title_FAC,...,TC__ENTERTAINMENT,TC__FOOD & DRINK,TC__HOME & LIVING,TC__OTHERS,TC__POLITICS,TC__QUEER VOICES,TC__SPORTS,TC__STYLE & BEAUTY,TC__TRAVEL,TC__WELLNESS
0,"""CNN Breaking News""",USA,1080369723227979776,2019-01-02 07:46:28,Winning ticket sold for $425 million New Year'...,"(CNN) Another year, another lottery you didn't...",0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,"""CNN Breaking News""",USA,1078295012532736001,2018-12-27 14:22:19,Clashes in Tunisia after journalist sets himse...,(CNN) Clashes in Tunisia have continued betwee...,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,"""CNN Breaking News""",USA,1079909388524154880,2019-01-01 01:17:16,"Chief Justice Roberts avoids controversy, prai...",(CNN) — Chief Justice John Roberts steered cle...,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,"""CNN Breaking News""",USA,1081296207790538753,2019-01-04 21:08:00,Mueller grand jury gets 6 month extension,Washington (CNN) Special counsel Robert Muelle...,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,"""CNN Breaking News""",USA,1081777463351541762,2019-01-06 05:00:20,Jazmine Barnes shooting: Police interview 'per...,(CNN) Detectives investigating the drive-by mu...,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
